In [ ]:
# notebooks/colab.ipynb
import os
# Change to the MILS_HW2 directory first
os.chdir('../MILS_HW2')

# Cell 1: Setup and downloads
%pip install -r requirements.txt
# Download all datasets
%python scripts/download_imagenette_cls.py
%python scripts/download_coco_det.py
%python scripts/download_voc_seg.py

In [1]:
# Cell 2: Verify downloads
# Add a verification step
print("Verifying downloads...")
import os
data_paths = {
    'seg': './data/VOCdevkit/VOC2012',
    'det': './data/coco_subset',
    'cls': './data/imagenette2-160'
}

for task, path in data_paths.items():
    if os.path.exists(path):
        print(f"{task} dataset found at {path}")
    else:
        print(f"WARNING: {task} dataset not found at {path}")

Verifying downloads...
seg dataset found at ./data/VOCdevkit/VOC2012
det dataset found at ./data/coco_subset
cls dataset found at ./data/imagenette2-160


In [2]:
# Cell 3: Model and Data initialization 
from src.models.unified_model import UnifiedModel
from src.datasets.data_loaders import create_dataloaders
from configs.config import Config  # 使用Config類

# 初始化配置
config = Config()  # 創建Config實例，不是模組

# 初始化模型
model = UnifiedModel()
print(f"Total parameters: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")

# 創建數據載入器 (根據你的下載檔案)
print("Loading datasets...")
dataloaders = create_dataloaders(
    batch_size=config.batch_size,
    num_workers=config.num_workers
)
print("dataloaders", dataloaders)

# 準備datasets字典給trainer使用 (只用train set)
datasets = {
    'seg': dataloaders['seg']['train'],
    'seg_val': dataloaders['seg']['val'],
    'det': dataloaders['det']['train'],
    'det_val': dataloaders['det']['val'],
    'cls': dataloaders['cls']['train'],
    'cls_val': dataloaders['cls']['val']
}
print("datasets:\n", datasets)
print("Datasets loaded successfully!")
print(f"Detection batches: {len(datasets['det'])}")
print(f"Segmentation batches: {len(datasets['seg'])}")
print(f"Classification batches: {len(datasets['cls'])}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total parameters: 3.2M
Loading datasets...
loading annotations into memory...
Done (t=10.60s)
creating index...
index created!
loading annotations into memory...
Done (t=1.23s)
creating index...
index created!
dataloaders {'seg': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da63f70>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da630a0>}, 'cls': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da63640>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da63370>}, 'det': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da631c0>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da62ef0>}}
datasets:
 {'seg': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da63f70>, 'seg_val': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da630a0>, 'det': <torch.utils.data.dataloader.DataLoader object at 0x7fed2da631c0>, 'det_val': <torch.utils.data.dataloader.DataLoader object at 0

In [9]:
# Cell 4: Three-stage training
from src.training.trainer import MultiTaskTrainer
trainer = MultiTaskTrainer(model, datasets, config)

# Stage 1: Segmentation baseline
# trainer.train_stage_1_segmentation()

# Debug: Check dataloader lengths and try to fetch a batch
print('Detection train batches:', len(datasets['det']))
print('Classification train batches:', len(datasets['cls']))

# Try to fetch a batch from detection
try:
    det_batch = next(iter(datasets['det']))
    print('First detection batch:', det_batch)
except Exception as e:
    print('Could not fetch detection batch:', e)
    det_batch = None

# Try to fetch a batch from classification
try:
    cls_batch = next(iter(datasets['cls']))
    print('First classification batch:', cls_batch)
except Exception as e:
    print('Could not fetch classification batch:', e)
    cls_batch = None
    
print('Type of datasets[\'det\']:', type(datasets['det']))
print('Type of datasets[\'cls\']:', type(datasets['cls']))
    
# Stage 2: Detection with EWC
if det_batch is not None and len(datasets['det']) > 0:
    trainer.train_stage_2_detection()
else:
    print('Skipping Stage 2: Detection dataloader is empty or invalid.')

# Stage 3: Classification with replay
if cls_batch is not None and len(datasets['cls']) > 0:
    trainer.train_stage_3_classification()
else:
    print('Skipping Stage 3: Classification dataloader is empty or invalid.')

# Validate forgetting constraint
success = trainer.validate_forgetting_constraint()
print(f"Forgetting constraint satisfied: {success}")

Detection train batches: 15
Classification train batches: 15
First detection batch: {'image': tensor([[[[ 0.7419,  0.7591,  0.7762,  ...,  1.1872,  1.1700,  1.1358],
          [ 0.7591,  0.7762,  0.8104,  ...,  1.2899,  1.2385,  1.1872],
          [ 0.7762,  0.7933,  0.8276,  ...,  1.3927,  1.3242,  1.2728],
          ...,
          [-0.5596, -0.5767, -0.5596,  ..., -1.5014, -1.5185, -1.5357],
          [-0.5596, -0.5596, -0.5767,  ..., -1.4329, -1.4158, -1.3815],
          [-0.5767, -0.5767, -0.5938,  ..., -1.2959, -1.2617, -1.2274]],

         [[ 1.0630,  1.0630,  1.0980,  ...,  1.4482,  1.4132,  1.3782],
          [ 1.0980,  1.0980,  1.1331,  ...,  1.5357,  1.4832,  1.4307],
          [ 1.1155,  1.1331,  1.1506,  ...,  1.6408,  1.5707,  1.5182],
          ...,
          [-0.4426, -0.4426, -0.4251,  ..., -1.4055, -1.4580, -1.4755],
          [-0.4426, -0.4251, -0.4426,  ..., -1.3004, -1.2829, -1.2479],
          [-0.4601, -0.4426, -0.4601,  ..., -1.1604, -1.1253, -1.1078]],

        

RuntimeError: size mismatch (got input: [16, 21, 112, 112] , target: [16, 224, 224]

In [ ]:
# Cell 5: Final evaluation
!python scripts/eval.py --weights checkpoints/final_model.pt --dataroot data --tasks all